In [1]:
%load_ext autoreload
%autoreload 2

In [61]:
import numpy as np
from timer import Timer
import pickle

In [64]:
maxFloat16 = np.finfo(np.float16).max
neuronMax = 100
neuronMin = -100
neuronDtype = np.float32


class NeuralNetwork:
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def sigmoidDerivative(x):
        return x * (1 - x)

    @staticmethod
    def relu(x):
        return np.maximum(0, x)

    @staticmethod
    def reluDerivative(x):
        return 1.0 * (x > 0)

    @staticmethod
    def softmax(x):
        exps = np.exp(x - np.max(x))
        return exps / np.sum(exps, axis=1, keepdims=True)

    @staticmethod
    def softmaxDerivative(x):
        return x * (1 - x)

    def __init__(self, layerSizes, activationFunctions):
        self.layerSizes = layerSizes
        self.activationFunctions = activationFunctions
        self.layers = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i]).astype(neuronDtype)
            for i in range(len(layerSizes))
        ]
        self.weights = [
            np.random.uniform(
                low=neuronMin, high=neuronMax, size=layerSizes[i] * layerSizes[i + 1]
            ).astype(neuronDtype).reshape(layerSizes[i], layerSizes[i + 1])
            for i in range(len(layerSizes) - 1)
        ]
        self.biases = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i + 1]).astype(
                neuronDtype
            )
            for i in range(len(layerSizes) - 1)
        ]

    def loadInput(self, input):
        self.layers[0] = input.astype(neuronDtype)

    def readOutput(self):
        return self.layers[-1]

    def forward(self):
        for i in range(len(self.weights)):
            self.layers[i + 1] = self.activationFunctions[i](
                np.dot(self.layers[i], self.weights[i]) + self.biases[i]
            )

    def backpropagate(self, target, learningRate):
        errors = [None] * len(self.layers)
        errors[-1] = target - self.layers[-1]
        for i in range(len(self.layers) - 2, 0, -1):
            errors[i] = errors[i + 1].dot(self.weights[i].T)

In [65]:
x = NeuralNetwork([30_000, 20, 20, 3], [NeuralNetwork.relu, NeuralNetwork.relu, NeuralNetwork.relu])
nnTimer = Timer(precision=3)


for i in range(1):
    nnTimer.tic()
    x.loadInput(np.random.rand(30_000))
    x.forward()
    nnTimer.toc()
    print(x.readOutput())
nnTimer.stats()

[0.0000000e+00 2.7870186e+08 4.5859564e+07]
Total time: 0 s
Total calls: 1
Average time: 0 ms


In [66]:
x.backpropagate(np.random.rand(3), 0.1)

In [59]:
np.finfo(np.float32).max

3.4028235e+38

In [62]:
pickle.dump(x, open("nn.pkl", "wb"))